In [1]:
## VISSIM Modules
import win32com.client as com
import os

## RL Modules
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("ERROR: GPU DEVICE NOT FOUND.")

from keras.models import load_model
    
## Data Management Modules
import pickle

## User Defined Modules
import Simulator_Functions as SF
from RLAgents import DQNAgent
from NParser import NetworkParser
from COMServer import COMServerDispatch, COMServerReload
from TupleToList import toList
from Utilities import log_progress, pltlive
## Other Modules
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

Default GPU Device: /device:GPU:0


Using TensorFlow backend.


In [2]:
## RL Hyperparamenters
# Number of simulations, save every "n" episodes and copy weights with frequency "f"
episodes = 500
partial_save_at = 100
copy_weights_frequency = 5

# Timesteps per simulation (1 timestep = 0.1 sec), length for random population is a multiple of episode
timesteps_per_second = 1
seconds_per_green = 6
seconds_per_yellow = 3
simulation_length = 3600*1 + 1

## State-Action Parameters
action_type = "phases"        # options are "phases" and "programs"
state_size = 4
action_size = 8

# Hyperparameters
PER_activated = True
batch_size = 256
memory_size = 1000
alpha   = 0.001
gamma   = 0.95
memory_population_length = 10*memory_size

# Exploration Schedule ("linear" or "geometric")
exploration_schedule = "geometric"
epsilon_start = 1
epsilon_end   = 0.001
if exploration_schedule == "linear":
    epsilon_decay = 1.2*(epsilon_end - epsilon_start)/(episodes-1)
    epsilon_sequence = [1 + epsilon_decay * entry for entry in range(episodes+1)]
    epsilon_sequence = [0 if entry < 0 else entry for entry in epsilon_sequence]
elif exploration_schedule == "geometric":
    epsilon_decay = np.power(epsilon_end/epsilon_start, 1./(episodes-1)) # Geometric decay
    epsilon_sequence = [epsilon_start * epsilon_decay ** entry for entry in range(episodes+1)]
else:
    print("ERROR: Unrecognized choice of exploration schedule.")

# Demand Schedule (times in seconds, demand in cars/hour as PPP)
demand_change_timesteps = 450
demand = {"h":800, 'm':400, 'l':200}
demand_list = [[demand['l'], demand['l']], [demand['m'], demand['l']],\
              [demand['h'], demand['l']], [demand['h'], demand['m']],\
              [demand['h'], demand['h']], [demand['m'], demand['h']],
              [demand['l'], demand['h']], [demand['l'], demand['m']]]

In [3]:
## Operation mode (selects functionalities)
mode = "debug"
# "populate" = population of memory, generation of initial memory file
# "training" = training agents, maximum speed, frozen UI, mid amount of messages
# "debug"    = trains for 1 episode, minimum speed, working UI, all messages
# "demo"     = loads pretrained agent, minimum speed, working UI
# "test"     = executes evaluation, maximum speed

## Network Model Parameters

model_name  = 'Balance'
# 'Single_Cross_Straight'
# 'Single_Cross_Triple'
# 'Balance'

vissim_working_directory = 'C:\\Users\\acabrejasegea\\OneDrive - The Alan Turing Institute\\Desktop\\ATI\\0_TMF\\MLforFlowOptimisation\\Vissim\\'
agent_type = 'DuelingDDQN'         # DQN, DuelingDQN, DDQN, DuelingDDQN
reward_type = 'Queues'   
# 'Queues'          Sum of the queues for all lanes in intersection
# 'QueuesDiff'      Difference in queue lengths in last timestep
# 'QueuesDiffSC'    10000* QueuesDiff - Queues^2
# 'TotalDelayDiff'
state_type  = 'Queues'    
Random_Seed = 42

## Use of additional files?
flag_read_additionally  = False
SaveResultsAgent = True
# Random demand
Random_Demand = False

# Session ID
Session_ID = 'Ep_'+str(episodes)+'_A_'+agent_type+"_Act_"+action_type+"_Rew_"+reward_type

In [4]:
# Initialize simulation
Vissim, Simulation, Network, cache_flag = COMServerDispatch(model_name, vissim_working_directory,\
                                                            memory_population_length, timesteps_per_second,\
                                                            delete_results = True, verbose = True)


Working Directory set to: C:\Users\acabrejasegea\OneDrive - The Alan Turing Institute\Desktop\ATI\0_TMF\MLforFlowOptimisation\Vissim\
Generating Cache...
Cache generated.

****************************
*   COM Server dispatched  *
****************************

Attempting to load Model File: Balance.inpx ...
Load process successful
Simulation length set to 10000 seconds.
Results from Previous Simulations: Deleted. Fresh Start Available.
Fetched and containerized Simulation Object
Fetched and containerized Network Object 

*******************************************************
*                                                     *
*                 SETUP COMPLETE                      *
*                                                     *
*******************************************************



In [5]:
npa = NetworkParser(Vissim)

In [6]:
Agents = [DQNAgent(state_size, action_size, ID, state_type, npa, memory_size,\
                           gamma, epsilon_sequence[0], alpha, copy_weights_frequency, Vissim, PER_activated,\
                           DoubleDQN = True if agent_type == "DDQN" or "DuelingDDQN" else False,\
                           Dueling = False if agent_type == "DQN" or "DDQN" else True) for ID in npa.signal_controllers_ids] 
time_t = 0

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Deploying instance of Double Deep Q Learning Agent(s)
Deploying instance of Double Deep Q Learning Agent(s)
Deploying instance of Double Deep Q Learning Agent(s)
Deploying instance of Double Deep Q Learning Agent(s)
Deploying instance of Double Deep Q Learning Agent(s)
Deploying instance of Double Deep Q Learning Agent(s)
Deploying instance of Double Deep Q Learning Agent(s)
Deploying instance of Double Deep Q Learning Agent(s)
Deploying instance of Double Deep Q Learning Agent(s)
Deploying instance of Double Deep Q Learning Agent(s)
Deploying instance of Double Deep Q Learning Agent(s)
Deploying instance of Double Deep Q Learning Agent(s)
Deploying instance of Double Deep Q Learning Agent(s)
Deploying instance of Double Deep Q Learning Agent(s)


In [ ]:
# Cycle through all agents and update them
Agents = SF.Agents_update(Agents, Vissim, state_type, reward_type, state_size, seconds_per_green, seconds_per_yellow, mode, 
time_t)

# Advance the game to the next second (proportionally to the simulator resolution).
for _ in range(0, timesteps_per_second):
    Vissim.Simulation.RunSingleStep()
    time_t += 1

In [ ]:
signal_controllers     = toList(Vissim.Net.SignalControllers.GetAll())
signal_controllers_ids = range(0,len(signal_controllers))
signal_groups = [[] for _ in signal_controllers_ids]

In [ ]:
print(signal_controllers)
print(len(signal_controllers))
print(signal_controllers_ids)
print(signal_groups)

In [ ]:
for SC in signal_controllers_ids:
    for SG in range(1,signal_controllers[SC].SGs.Count+1):
        signal_groups[SC].append(signal_controllers[SC].SGs.ItemByKey(SG))

In [ ]:
## SC01: Left Middle of the MAP. Four Way intersection. 12 signal groups. Identified by direction
##       of incoming traffic/
# 1 : East. Turn right and straight.
# 2 : North. All directions.
# 3 : West. Only turn left.
# 4 : West. Turn right and straight.
# 5 : South. All directions.
# 6 : Pedestrian located in South East.
# 7 : Pedestrian located in North East.
# 8 : Pedestrian located in North.
# 9 : Pedestrian located in North West.
# 10: Pedestrian located in South West.
# 11: Pedestrian located in South.
# 12: East. Only turn left.

model_phases_SC01 = [[1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0],\
                     [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1],\
                     [0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0]]

extra_phases_SC01 = [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],\
                     [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]

model_phases_SC02 = [[0, 1, 0, 0, 1, 0, 1, 1],\
                     [1, 0, 0, 1, 0, 0, 0, 0],\
                     [0, 0, 1, 0, 0, 1, 0, 0]]

extra_phases_SC02 = [[1, 1, 0, 0, 0, 0, 0, 0],\
                     [0, 0, 0, 1, 1, 0, 0, 0]]

model_phases_SC03 = [[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],\
                     [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],\
                     [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],\
                     [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

extra_phases_SC03 = [[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],\
                     [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],\
                     [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],\
                     [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]

model_phases_SC04 = [[0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0],\
                     [1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0],\
                     [0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1]]

extra_phases_SC04 = [[1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
                     [0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0]]

model_phases_SC05 = [[0, 1, 1, 0, 0],\
                     [1, 1, 0, 0, 0],\
                     [0, 0, 0, 1, 0]]

# No extra phases for SC05

model_phases_SC06 = [[1, 0, 1, 0, 1, 0],\
                     [0, 1, 0, 1, 0, 1]]

# No extra phases for SC06

model_phases_SC07 = [[0, 1, 0, 1, 1, 1],
                     [1, 0, 1, 0, 0, 0]]

# No extra phases for SC07

model_phases_SC08 = [[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0],\
                     [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],\
                     [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

extra_phases_SC08 = [[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],\
                     [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]]

model_phases_SC09 = [[1, 0, 1, 0, 0, 0, 0],\
                     [0, 1, 0, 0, 0, 0, 0]]

# No extra phases for SC09

model_phases_SC10 = [[0, 1, 0, 1, 1, 0, 1, 0],\
                     [1, 0, 1, 0, 0, 1, 0, 1]]

# No extra phases for SC10

model_phases_SC11 = [[1, 0, 1, 0, 0, 1, 0, 1],\
                     [0, 1, 0, 1, 1, 0, 1, 0]]

# No extra phases for SC11

model_phases_SC12 = [[1, 0, 1, 0, 0, 1, 0, 1],\
                     [0, 1, 0, 1, 1, 0, 1, 0]]

# No extra phases for SC12

model_phases_SC13 = [[1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1],\
                     [0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1],\
                     [0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0],\
                     [0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0]]

# No extra pahses for SC13

model_phases_SC14 = [[1, 0, 0, 1, 0, 0, 1, 0, 0],\
                     [0, 0, 1, 1, 0, 1, 0, 0, 0],\
                     [0, 1, 0, 0, 1, 1, 0, 1, 1]]

# No extra phases for SC14

In [ ]:
for i in range(10):
    Vissim.Simulation.RunSingleStep()

In [ ]:
signal_groups[1][7].SetAttValue("SigState", "AMBER")

In [ ]:
signal_controllers[13].SGs.ItemByKey(1).SetAttValue("SigState", "AMBER")
for i in range(10):
    Vissim.Simulation.RunSingleStep()

In [ ]:
for index, SC in enumerate(signal_controllers):
    print(index)
    for SG in range(1,SC.SGs.Count):
        signal_groups[index].append(SC.SGs.ItemByKey(SG+1))

In [ ]:
a = toList(signal_groups[0][0].SigHeads.GetAll())
print(a)
print(signal_controllers)

In [ ]:
## Create SignalHeadsCollection and unpack the SignalHeads into a list by SignalController
signal_heads = [[] for _ in signal_controllers_ids]
for SC in signal_controllers_ids:
    print("Signal Controller {}".format(SC))
    print("Total of {} Signal Groups".format(signal_controllers[SC].SGs.Count))
    for SG in range(signal_controllers[SC].SGs.Count):
        print("Processing Signal Group {}".format(SG))
        signal_heads[SC].append(toList(signal_groups[SC][SG].SigHeads.GetAll())[0])

In [ ]:
toList(signal_groups[2][8].SigHeads.GetAll())

In [ ]:
for SC in signal_controllers[0:15]:
    for SG in range(1, SC.SGs.Count+1):
        SC.SGs.ItemByKey(SG).SetAttValue("SigState", "GREEN")
for i in range(10):
    Vissim.Simulation.RunSingleStep()

In [ ]:
for SC in signal_controllers:
    print(SC.SGs.Count)

In [ ]:
signal_groups = [[] for _ in signal_controllers_ids]
for SC in signal_controllers_ids:
    for SG in range(1,signal_controllers[SC].SGs.Count+1):
        signal_groups[SC].append(signal_controllers[SC].SGs.ItemByKey(SG))
        
for SC in signal_controllers_ids:
    print(len(signal_groups[SC]))

In [ ]:
signal_groups[2][8]

In [7]:
a = [[1],[2]]
b = [[3], [4]]
c = a+b
c

[[1], [2], [3], [4]]